In [ ]:
!pip install selenium
!apt update
!apt install chromium-chromedriver

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
51 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
c

In [ ]:
# Necessary Imports

import os
import re
import csv
import pandas as pd
from random import uniform
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
import math
from random import uniform
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
import pandas as pd
import os
import csv


In [ ]:
# Chromium Headless Options

chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--remote-debugging-port=9222")

driver = webdriver.Chrome(options=chrome_options)

SessionNotCreatedException: Message: session not created: Chrome failed to start: exited normally.
  (chrome not reachable)
  (The process started from chrome location /root/.cache/selenium/chrome/linux64/126.0.6478.126/chrome is no longer running, so ChromeDriver is assuming that Chrome has crashed.)
Stacktrace:
#0 0x5601f38ebc7a <unknown>
#1 0x5601f35cee2c <unknown>
#2 0x5601f3603d6a <unknown>
#3 0x5601f360017b <unknown>
#4 0x5601f364ac49 <unknown>
#5 0x5601f363e363 <unknown>
#6 0x5601f360e247 <unknown>
#7 0x5601f360eb9e <unknown>
#8 0x5601f38b224b <unknown>
#9 0x5601f38b62f1 <unknown>
#10 0x5601f389dafe <unknown>
#11 0x5601f38b6e52 <unknown>
#12 0x5601f388279f <unknown>
#13 0x5601f38db638 <unknown>
#14 0x5601f38db810 <unknown>
#15 0x5601f38eadac <unknown>
#16 0x7d7134c74ac3 <unknown>


# Flipkart ( https://www.flipkart.com/ )

In [ ]:
!mkdir flipkart

In [ ]:
# Chromium Headless Options

chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--remote-debugging-port=9222")

driver = webdriver.Chrome(options=chrome_options)

with open("flipkart_urls.csv", mode="r") as file:
    csv_reader = csv.DictReader(file)
    for row in csv_reader:
        baseUrl = row["URL"]
        product_fullName = baseUrl.split("/")[3]
        product_id = baseUrl.split("/")[5].split("?")[0]
        product_url = ("https://www.flipkart.com/" + product_fullName + "/product-reviews/" + product_id + "?page={}")

        reviews_dict = {
            "product_id": [],
            "product_fullName": [],
            "product_url": [],
            "product_name": [],
            "product_cost": [],
            "review_title": [],
            "review_content": [],
            "review_rating": [],
            "review_date": [],
            "review_location": [],
            "review_likes": [],
            "review_dislikes": [],
        }

        start_page = 1
        end_page = None

        try:
            if end_page is None:
                driver.get(product_url.format(1))
                no_page = driver.find_element(By.CSS_SELECTOR, "._1G0WLw.mpIySA")
                page_html = BeautifulSoup(no_page.get_attribute("outerHTML"), "html.parser")
                page_content = (
                    page_html.find("div", class_="_1G0WLw mpIySA")
                    .find_all("span")[0]
                    .get_text()
                    .split(" ")[-1]
                )
                end_page = int(page_content)
        except Exception as e:
            print(e)
            end_page = 1

        try:
            for j in range(start_page, end_page + 1):

                driver.get(product_url.format(j))

                content_div = driver.find_element(By.CSS_SELECTOR, ".DOjaWF.gdgoEp.col-9-12")

                content_html = content_div.get_attribute("outerHTML")
                result = BeautifulSoup(content_html, "html.parser")

                reviews = result.find_all("div", class_="cPHDOP col-12-12")

                product_name = " ".join(result.find("div", class_="Vu3-9u eCtPz5").contents[0].strip().split(" ")[:2])

                product_cost = driver.find_element(By.CLASS_NAME, "yRaY8j").text.replace("₹", "")

                for i in reviews:
                    main = i.find("div", class_="col EPCmJX Ma1fCG")

                    extract_review_title = lambda i: (
                        i.find("p", class_="z9E0IG").get_text().strip()
                        if i.find("p", class_="z9E0IG")
                        else None
                    )

                    if extract_review_title(i) == None:
                        continue

                    extract_review_content = lambda i: (
                        re.sub(r"\s+", " ", i.find("div", class_="ZmyHeo").get_text())
                        .replace("READ MORE", "")
                        .strip()
                        if i.find("div", class_="ZmyHeo")
                        else " "
                    )

                    extract_review_rating = lambda i: (
                        i.find("div", class_="XQDdHH Ga3i8K").get_text().strip().split()[0]
                        if i.find("div", class_="XQDdHH Ga3i8K")
                        else " "
                    )

                    extract_review_date = lambda i: (
                        i.find_all("p", class_="_2NsDsF")[1].get_text().strip()
                        if i.find_all("p", class_="_2NsDsF")[1]
                        else " "
                    )

                    extract_review_location = lambda i: (
                        i.find("p", class_="MztJPv").get_text().strip().split(",")[1]
                        if i.find("p", class_="MztJPv")
                        else " "
                    )

                    extract_likes_dislikes = lambda i: [
                        i.find_all("span", class_="tl9VpF")[0].get_text().strip(),
                        i.find_all("span", class_="tl9VpF")[1].get_text().strip(),
                    ]

                    review_title = extract_review_title(i)
                    review_content = extract_review_content(i)
                    review_rating = extract_review_rating(i)
                    review_date = extract_review_date(i)
                    review_location = extract_review_location(i)
                    review_likes = extract_likes_dislikes(i)[0]
                    review_dislikes = extract_likes_dislikes(i)[1]

                    reviews_dict["product_id"].append(product_id)
                    reviews_dict["product_fullName"].append(product_fullName)
                    reviews_dict["product_url"].append(product_url.format(j))
                    reviews_dict["product_name"].append(product_name)
                    reviews_dict["product_cost"].append(product_cost)
                    reviews_dict["review_title"].append(review_title)
                    reviews_dict["review_content"].append(review_content)
                    reviews_dict["review_rating"].append(review_rating)
                    reviews_dict["review_date"].append(review_date)
                    reviews_dict["review_location"].append(review_location)
                    reviews_dict["review_likes"].append(review_likes)
                    reviews_dict["review_dislikes"].append(review_dislikes)

        finally:
            df = pd.DataFrame(reviews_dict)
            while True:
                if os.path.isfile(f"./flipkart/{product_id}.csv"):
                    product_id = product_id + "-" + str(int(uniform(1, 100)))
                else:
                    break
            csv_filename = f"./flipkart/{product_id}.csv"
            df.to_csv(csv_filename, index=False)
            print(f"Data saved to {csv_filename}")

https://www.flipkart.com/intel-i7-3770-3-9-ghz-lga-1155-socket-4-cores-desktop-processor/product-reviews/itmfa5j4jcma4px6?page=1
Data saved to ./flipkart/itmfa5j4jcma4px6.csv


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"._1G0WLw.mpIySA"}
  (Session info: chrome-headless-shell=126.0.6478.126); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
#0 0x58f60fc28c7a <unknown>
#1 0x58f60f90be2c <unknown>
#2 0x58f60f958661 <unknown>
#3 0x58f60f958751 <unknown>
#4 0x58f60f99cf64 <unknown>
#5 0x58f60f97b5ed <unknown>
#6 0x58f60f99a303 <unknown>
#7 0x58f60f97b363 <unknown>
#8 0x58f60f94b247 <unknown>
#9 0x58f60f94bb9e <unknown>
#10 0x58f60fbef24b <unknown>
#11 0x58f60fbf32f1 <unknown>
#12 0x58f60fbdaafe <unknown>
#13 0x58f60fbf3e52 <unknown>
#14 0x58f60fbbf79f <unknown>
#15 0x58f60fc18638 <unknown>
#16 0x58f60fc18810 <unknown>
#17 0x58f60fc27dac <unknown>
#18 0x7e104114dac3 <unknown>


In [ ]:
!rm -r flipkart
!mkdir flipkart

# CPU UserBenchmarks ( https://cpu.userbenchmark.com/ )

In [ ]:
!mkdir cpu-userbench

In [ ]:
with open("userbench_urls.csv", mode="r") as file:
    csv_reader = csv.DictReader(file)
    for row in csv_reader:
        baseUrl = row["URL"]
        product_name = baseUrl.split("/")[3]
        product_id = baseUrl.split("/")[5].split("?")[0]

        reviews_dict = {
            "product_id": [],
            "product_url": [],
            "product_name": [],
            "review_title": [],
            "review_content": [],
            "review_date": [],
        }

        driver.get(baseUrl)
        print(driver.title)
        try:

            content_div = driver.find_element(By.ID, "wikiForm").get_attribute("outerHTML")
            page_html = BeautifulSoup(content_div, "html.parser")
            page_content = page_html.find_all("div", class_="row showchildlinkonhover")
            for i in page_content:
                review_title = ""
                review_date = ""
                review_content = ""
                content = i.find("div", class_="col-xs-9 fancyfont")
                if content:
                    review_header = content.find("h4", class_="lighterblacktext")
                    if review_header:
                        review_title = review_header.get_text()

                    review_date = content.find("span", class_="wiki-ts")
                    if review_date:
                        review_date = review_date.get_text()

                    review_content = content.find("p", class_="medp")
                    if review_content:
                        review_content = review_content.get_text().strip()
                        review_content = " ".join(review_content.split())

                reviews_dict["product_id"].append(product_id)
                reviews_dict["product_url"].append(baseUrl)
                reviews_dict["product_name"].append(product_name)
                reviews_dict["review_title"].append(review_title)
                reviews_dict["review_content"].append(review_content)
                reviews_dict["review_date"].append(review_date)


        finally:
            df = pd.DataFrame(reviews_dict)
            # print(df)
            while True:
                if os.path.isfile(f"./cpu-userbench/{product_id}.csv"):
                    product_id = product_id + "-" + str(int(uniform(1, 100)))
                else:
                    break
            csv_filename = f"./cpu-userbench/{product_id}.csv"
            df.to_csv(csv_filename, index=False)
            print(f"Data saved to {csv_filename}")


UserBenchmark: Intel Core i5-12600K BX8071512600K
Data saved to ./cpu-userbench/4120.csv
UserBenchmark: Intel Core i5-13600K BX8071513600K
Data saved to ./cpu-userbench/4134.csv
UserBenchmark: Intel Core i5-12400F BX8071512400F
Data saved to ./cpu-userbench/4121.csv


KeyboardInterrupt: 

# Extract Product Names

In [ ]:
df_name = pd.read_csv('product_names.csv')

In [ ]:
# Chromium Headless Options

chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--remote-debugging-port=9222")

driver = webdriver.Chrome(options=chrome_options)
pattern = (r"https://www\.intel\.com/content/www/us/en/products/sku/.+/specifications\.html")


names_dict = {"product_name": [], "product_fullName": []}

with open("product_names.csv", mode="r") as file:
    csv_reader = csv.DictReader(file)
    for row in csv_reader:
        baseUrl = row["URL"]
        google_search = "https://www.google.com/search?q=" + baseUrl

        driver.get(google_search)
        content_div = driver.find_element(By.ID, "rcnt").get_attribute("outerHTML")
        page_html = BeautifulSoup(content_div, "html.parser")
        a_tags = page_html.find_all("a")

        for tag in a_tags:
            url = tag.get("href")
            if(url is None):
                continue
            match = re.match(pattern, url)
            if match:
                driver.get(url)
                processor_html = driver.find_element(
                    By.ID, "specs-1-0-0"
                ).get_attribute("outerHTML")
                processor_bs4 = BeautifulSoup(processor_html, "html.parser")
                a = processor_bs4.find_all(
                    "div", class_="col-xs-6 col-lg-6 tech-data"
                )
                product_name = a[3].get_text().strip()

                print(product_name)

                names_dict["product_name"].append(product_name)
                names_dict["product_fullName"].append(baseUrl)

        driver.implicitly_wait(5)


df = pd.DataFrame(names_dict)
while True:
    if os.path.isfile(f"./product_names/{baseUrl}.csv"):
        baseUrl = baseUrl + "-" + str(int(uniform(1, 100)))
    else:
        break
csv_filename = f"./product_names/{baseUrl}.csv"
df.to_csv(csv_filename, index=False)
print(f"Data saved to {csv_filename}")


# Amazon

In [ ]:
!mkdir amazon

In [ ]:
# Chromium Headless Options

chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--remote-debugging-port=9222")

driver = webdriver.Chrome(options=chrome_options)


with open("amazon_urls.csv", mode="r") as file:
    csv_reader = csv.DictReader(file)
    for row in csv_reader:
        baseUrl = row["URL"]

        product_id = baseUrl.split("/")[5]
        product_review_url = (
            "https://www."
            + "amazon.in"
            + "/dp/product-reviews/"
            + product_id
            + "?pageNumber={}"
        )

        reviews_dict = {
            "product_id": [],
            "product_fullName": [],
            "product_review_url": [],
            "product_url": [],
            "product_name": [],
            "product_cost": [],
            "product_average_rating": [],
            "review_title": [],
            "review_content": [],
            "review_rating": [],
            "review_date": [],
            "review_location": [],
        }

        start_page = 8
        end_page = None

        try:
            if end_page is None:
                driver.get(product_review_url.format(1))
                no_reviews = driver.find_element(
                    By.XPATH, '//div[@data-hook="cr-filter-info-review-rating-count"]'
                )
                end_page = math.ceil(
                    int(no_reviews.text.split(" ")[3].replace(",", "")) / 10
                )
        except Exception as e:
            print(f"Error determining the number of pages: {e}")
            end_page = 1

        try:
            for j in range(start_page, end_page + 1):
                driver.get(product_review_url.format(j))

                product_fullName = driver.find_element(
                    By.CSS_SELECTOR, 'a[data-hook="product-link"]'
                ).text
                product_url = driver.find_element(
                    By.CSS_SELECTOR, 'a[data-hook="product-link"]'
                ).get_attribute("href")
                product_average_rating = driver.find_element(
                    By.CSS_SELECTOR, 'span[data-hook="rating-out-of-text"]'
                ).text.split(" ")[0]

                try:
                    shall_break = driver.find_element(
                        By.CSS_SELECTOR, ".a-disabled.a-last"
                    ).text

                except NoSuchElementException:
                    shall_break = False

                print(f"Scraping page: {product_review_url.format(j)}")
                content_div = driver.find_element(
                    By.CSS_SELECTOR, ".a-section.a-spacing-none.review-views.celwidget"
                )
                content_html = content_div.get_attribute("outerHTML")
                result = BeautifulSoup(content_html, "html.parser")

                for review_div in result.find_all("div", {"data-hook": "review"}):
                    title, review, attribute, rating, date, country = (
                        None,
                        None,
                        None,
                        None,
                        None,
                        None,
                    )

                    title_span = review_div.find("a", {"data-hook": "review-title"})
                    if title_span:
                        title = title_span.find_all("span")[-1].text.strip()

                    attribute_span = review_div.find(
                        "span", {"class": "a-profile-name"}
                    )
                    if attribute_span:
                        attribute = attribute_span.text.strip()

                    rating_span = review_div.find(
                        "i", {"data-hook": "review-star-rating"}
                    )
                    if rating_span:
                        rating = rating_span.find("span").text.strip().split(" ")[0]

                    date_span = review_div.find("span", {"data-hook": "review-date"})
                    if date_span:
                        date_text = date_span.text.strip()
                        date_parts = date_text.split(" ")
                        country = date_parts[2]
                        date = " ".join(date_parts[-3:])

                    review_span = review_div.find("span", {"data-hook": "review-body"})
                    if review_span:
                        review = review_span.text.strip()

                    reviews_dict["product_id"].append(product_id)
                    reviews_dict["product_fullName"].append(product_fullName)
                    reviews_dict["product_url"].append(product_url)
                    reviews_dict["product_name"].append(
                        (" ").join(product_fullName.split(" ")[:4])
                    )
                    reviews_dict["product_review_url"] = product_review_url.format(j)
                    reviews_dict["product_cost"].append("")
                    reviews_dict["review_title"].append(title)
                    reviews_dict["review_content"].append(review)
                    reviews_dict["review_rating"].append(rating)
                    reviews_dict["review_date"].append(date)
                    reviews_dict["review_location"].append(country)
                    reviews_dict["product_average_rating"].append(
                        product_average_rating
                    )

                if shall_break:
                    break

        finally:
            df = pd.DataFrame(reviews_dict)
            while True:
                if os.path.isfile(f"./amazon/{product_id}.csv"):
                    product_id = product_id + "-" + str(int(uniform(1, 100)))
                else:
                    break
            csv_filename = f"./amazon/{product_id}.csv"
            df.to_csv(csv_filename, index=False)
            print(f"Data saved to {csv_filename}")


SessionNotCreatedException: Message: session not created: Chrome failed to start: exited normally.
  (chrome not reachable)
  (The process started from chrome location /root/.cache/selenium/chrome/linux64/126.0.6478.126/chrome is no longer running, so ChromeDriver is assuming that Chrome has crashed.)
Stacktrace:
#0 0x598189bdbc7a <unknown>
#1 0x5981898bee2c <unknown>
#2 0x5981898f3d6a <unknown>
#3 0x5981898f017b <unknown>
#4 0x59818993ac49 <unknown>
#5 0x59818992e363 <unknown>
#6 0x5981898fe247 <unknown>
#7 0x5981898feb9e <unknown>
#8 0x598189ba224b <unknown>
#9 0x598189ba62f1 <unknown>
#10 0x598189b8dafe <unknown>
#11 0x598189ba6e52 <unknown>
#12 0x598189b7279f <unknown>
#13 0x598189bcb638 <unknown>
#14 0x598189bcb810 <unknown>
#15 0x598189bdadac <unknown>
#16 0x78f74939aac3 <unknown>
